# Binary classification of mushrooms with tensorflow : simple_model and GradCAM

## dans ce notebook :
- entrainement d'un modèle avec une architecture simple
- interprétation des résultats avec des GradCAM

Les inputs de ce notebook sont :
- un dossier d'images contenant l'ensemble des images disponibles
- un dataframe ou un fichier .csv contenant 2 colonnes :
    - la première 'filepath' avec les noms des fichiers images,
    - la seconde 'label' avec le nom des labels à identifier (0 = inedible, 1 = edible)

###
Ce notebook est inspiré du travail de HOA NGUYEN (https://www.kaggle.com/code/nguyenhoa/dog-cat-classifier-gradcam-with-tensorflow-2-0/notebook)

In [ ]:
# import des librairies nécessaires
import numpy as np
import os
import zipfile
import pandas as pd
import random
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split

# fonctionne avec tensorflow v2.9.1
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Flatten, Activation
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import InputLayer, Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras import backend as K
print("Tensorflow version: ", tf.__version__)

## Préparation des données 

In [ ]:
# chermin du dossier source et paramètres de base
IMAGE_DIR = r"C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\Dataset\Binary_Classification\reduced_dataset\input\full_reduced_dataset\train"
H = 224 # A adapter en fonction de l'input du modele à entrainer
W = 224 # A adapter en fonction de l'input du modele à entrainer
epochs = 20
batch_size = 128
SEED = 3

In [ ]:
# import du fichier .csv ou création du dataframe
df = pd.read_csv(r'C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\Dataset\Binary_Classification\reduced_dataset\input\full_reduced_dataset\train_labels.csv')
df.drop(['Unnamed: 0', 'kingdom', 'phylum', 'family', 'order', 'classes', 'genus', 'species'], axis = 1, inplace = True)
df.rename({'image_lien' : 'filename', 'edible' : 'label'}, axis = 1, inplace = True)
df.head()


In [ ]:
df.label.value_counts()

## Création des sets de données : train, validation, test
Le dataset initial est séparé en 3 parties :
- une partie test représentant 20% des données initiales (test_df)
- une partie train représentant 64 % des données initiales (train_df)
- une partie validation représentant 16% des données initiales (val_df)

Train, servira à l'entrainement, Val à l'évaluation durant la phase d'entrainement, Test à comparer les résultats issus de différents modèles.

In [ ]:
# Séparation du dataset initial en 2 parties (train et test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state = SEED, stratify = df.label)
train_df.sample(frac=1, random_state=SEED)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# Séparation du dataset train en 2 parties (train et val)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state = SEED, stratify = train_df.label)
train_df.sample(frac=1, random_state=SEED)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)



In [ ]:
# sauvegarde des différents dataframes
output_dir = os.path.dirname(IMAGE_DIR)

train_df.to_csv(f"{output_dir}/train_df.csv")
val_df.to_csv(f"{output_dir}/val_df.csv")
test_df.to_csv(f"{output_dir}/test_df.csv")

In [ ]:
# Visualisation des distributions des labels dans les différents jeux de données
dataframes = ('train_df', 'val_df', 'test_df')
counts = {
    'inedible' : np.array([train_df.label.value_counts()[1] / len(train_df), val_df.label.value_counts()[1] / len(val_df), test_df.label.value_counts()[1] / len(test_df)]),
    'edible' : np.array([train_df.label.value_counts()[0] / len(train_df), val_df.label.value_counts()[0] / len(val_df), test_df.label.value_counts()[0] / len(test_df)])
}
width = 0.5
fig, ax = plt.subplots()
bottom = np.zeros(3)
for boolean, counts in counts.items():
    p = ax.bar(dataframes, counts, width, label=boolean, bottom=bottom)
    bottom += counts
ax.set_title('Labels distribution for differents datasets')
ax.legend(loc='upper right')
plt.show()



### Rééquilibrage du jeu de données 

In [ ]:
# pour la partie train
label_counts = train_df.groupby('label').size()
train_df_X = train_df.groupby('label').apply(lambda x : x.sample(label_counts.min()))

# pour la partie val
label_counts = val_df.groupby('label').size()
val_df_X = val_df.groupby('label').apply(lambda x : x.sample(label_counts.min()))

# pour la partie test
label_counts = test_df.groupby('label').size()
test_df_X = test_df.groupby('label').apply(lambda x : x.sample(label_counts.min()))



# sauvegarde des différents dataframes
output_dir = os.path.dirname(IMAGE_DIR)

train_df_X.to_csv(f"{output_dir}/train_df_X.csv")
val_df_X.to_csv(f"{output_dir}/val_df_X.csv")
test_df_X.to_csv(f"{output_dir}/test_df_X.csv")



# Visualisation des distributions des labels dans les différents jeux de données rééquilibrés
dataframes = ('train_df_X', 'val_df_X', 'test_df_X')
counts = {
    'inedible' : np.array([train_df_X.label.value_counts()[1] / len(train_df_X), val_df_X.label.value_counts()[1] / len(val_df_X), test_df_X.label.value_counts()[1] / len(test_df_X)]),
    'edible' : np.array([train_df_X.label.value_counts()[0] / len(train_df_X), val_df_X.label.value_counts()[0] / len(val_df_X), test_df_X.label.value_counts()[0] / len(test_df_X)])
}
width = 0.5
fig, ax = plt.subplots()
bottom = np.zeros(3)
for boolean, counts in counts.items():
    p = ax.bar(dataframes, counts, width, label=boolean, bottom=bottom)
    bottom += counts
ax.set_title('Labels distribution for differents balanced datasets')
ax.legend(loc='upper right')
plt.show()

## Exploration de données 

In [ ]:
inedible = list(df[df.label==0].filename)
edible = list(df[df.label==1].filename)

In [ ]:
# création de fonctions permettant de générer une grille de 100 images avec des transformations (rotation) aléatoires
# code adaptés avec quelques modifications de
# https://www.kaggle.com/serkanpeldek/keras-cnn-transfer-learnings-on-cats-dogs-dataset

def get_side(img, side_type, n = 5):
    h, w, c = img.shape
    if side_type == "horizontal":
        return np.ones((h,n,c))
    return np.ones((n,w,c))

def show_gallery(im_ls,n=5, shuffle=True):
    images = []
    vertical_images = []
    if shuffle:
        random.shuffle(im_ls)
    vertical_images = []
    for i in range(n*n):
        img = load_img(os.path.join(IMAGE_DIR,im_ls[i]), target_size=(W,H))
        img = img_to_array(img)
        hside = get_side(img,side_type="horizontal")
        images.append(img)
        images.append(hside)
        
        if (i+1) % n == 0:
            himage=np.hstack((images))
            vside = get_side(himage, side_type="vertical")
            vertical_images.append(himage)
            vertical_images.append(vside)
            
            images = []
        
    gallery = np.vstack((vertical_images))
    plt.figure(figsize=(20,20))
    plt.axis("off")
    plt.imshow(gallery.astype(np.uint8))
    plt.show()

In [ ]:
# observation d'une grille d'images de champignons comestibles
show_gallery(edible, n=10)


In [ ]:
# observation d'une grille d'images de champignons non-comestibles
show_gallery(inedible, n=10)


## Définition des classes GradCAM & GuidedGradCAM 

### GradCAM

In [ ]:
class GradCAM:
    # Adapté avec quelques modifications de  https://www.pyimagesearch.com/2020/03/09/grad-cam-visualize-class-activation-maps-with-keras-tensorflow-and-deep-learning/
    def __init__(self, model, layerName=None):
        """
        model: pre-softmax layer (logit layer)
        """
        self.model = model
        self.layerName = layerName
            
        if self.layerName == None:
            self.layerName = self.find_target_layer()
    
    def find_target_layer(self):
        for layer in reversed(self.model.layers):
            if len(layer.output_shape) == 4:
                return layer.name
        raise ValueError("Could not find 4D layer. Cannot apply GradCAM")
            
    def compute_heatmap(self, image, classIdx, upsample_size, eps=1e-5):
        gradModel = Model(
            inputs = [self.model.inputs],
            outputs = [self.model.get_layer(self.layerName).output, self.model.output]
        )
        # record operations for automatic differentiation
        
        with tf.GradientTape() as tape:
            inputs = tf.cast(image, tf.float32)
            (convOuts, preds) = gradModel(inputs) # preds after softmax
            loss = preds[:,classIdx]
        
        # compute gradients with automatic differentiation
        grads = tape.gradient(loss, convOuts)
        # discard batch
        convOuts = convOuts[0]
        grads = grads[0]
        norm_grads = tf.divide(grads, tf.reduce_mean(tf.square(grads)) + tf.constant(eps))
        
        # compute weights
        weights = tf.reduce_mean(norm_grads, axis=(0,1))
        cam = tf.reduce_sum(tf.multiply(weights, convOuts), axis=-1)
        
        # Apply reLU
        cam = np.maximum(cam, 0)
        cam = cam/np.max(cam)
        cam = cv2.resize(cam, upsample_size,interpolation=cv2.INTER_LINEAR)
        
        # convert to 3D
        cam3 = np.expand_dims(cam, axis=2)
        cam3 = np.tile(cam3, [1,1,3])
        
        return cam3
    
def overlay_gradCAM(img, cam3):
    cam3 = np.uint8(255*cam3)
    cam3 = cv2.applyColorMap(cam3, cv2.COLORMAP_JET)
    
    new_img = 0.3*cam3 + 0.5*img
    
    return (new_img*255.0/new_img.max()).astype("uint8")


### GuidedGradCAM

In [ ]:
@tf.custom_gradient
def guidedRelu(x):
    def grad(dy):
        return tf.cast(dy>0,"float32") * tf.cast(x>0, "float32") * dy
    return tf.nn.relu(x), grad

# Reference: https://github.com/eclique/keras-gradcam with adaption to tensorflow 2.0  
class GuidedBackprop:
    def __init__(self,model, layerName=None):
        self.model = model
        self.layerName = layerName
        self.gbModel = self.build_guided_model()
        
        if self.layerName == None:
            self.layerName = self.find_target_layer()

    def find_target_layer(self):
        for layer in reversed(self.model.layers):
            if len(layer.output_shape) == 4:
                return layer.name
        raise ValueError("Could not find 4D layer. Cannot apply Guided Backpropagation")

    def build_guided_model(self):
        gbModel = Model(
            inputs = [self.model.inputs],
            outputs = [self.model.get_layer(self.layerName).output]
        )
        layer_dict = [layer for layer in gbModel.layers[1:] if hasattr(layer,"activation")]
        for layer in layer_dict:
            if layer.activation == tf.keras.activations.relu:
                layer.activation = guidedRelu
        
        return gbModel
    
    def guided_backprop(self, images, upsample_size):
        """Guided Backpropagation method for visualizing input saliency."""
        with tf.GradientTape() as tape:
            inputs = tf.cast(images, tf.float32)
            tape.watch(inputs)
            outputs = self.gbModel(inputs)

        grads = tape.gradient(outputs, inputs)[0]

        saliency = cv2.resize(np.asarray(grads), upsample_size)

        return saliency

def deprocess_image(x):
    # Same normalization as in:
    # https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    
    # normalize tensor: center on 0., ensure std is 0.25
    x = x.copy()
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.25

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

### Création d'une fonction de visualisation des GradCAM et GuidedGradCAM

In [ ]:
def show_gradCAMs(model, gradCAM, GuidedBP, im_ls, n=3, decode={}):
    """
    model: softmax layer
    """
    random.shuffle(im_ls)
    plt.subplots(figsize=(30, 10*n))
    k=1
    for i in range(n):
        img = cv2.imread(os.path.join(IMAGE_DIR,im_ls[i]))
        upsample_size = (img.shape[1],img.shape[0])
        if (i+1) == len(df):
            break
        # Show original image
        plt.subplot(n,3,k)
        plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
        plt.title("Filename: {}".format(im_ls[i]), fontsize=20)
        plt.axis("off")
        # Show overlayed grad
        plt.subplot(n,3,k+1)
        im = img_to_array(load_img(os.path.join(IMAGE_DIR,im_ls[i]), target_size=(W,H)))
        x = np.expand_dims(im, axis=0)
        x = preprocess_input(x)
        preds = model.predict(x)
        idx = preds.argmax()
        if len(decode)==0:
            res = decode_predictions(preds)[0][0][1:]
        else:
            res = [decode[idx],preds.max()]
        cam3 = gradCAM.compute_heatmap(image=x, classIdx=idx, upsample_size=upsample_size)
        new_img = overlay_gradCAM(img, cam3)
        new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
        plt.imshow(new_img)
        plt.title("GradCAM - Pred: {}. Prob: {}".format(res[0],res[1]), fontsize=20)
        plt.axis("off")
        
        # Show guided GradCAM
        plt.subplot(n,3,k+2)
        gb = GuidedBP.guided_backprop(x, upsample_size)
        guided_gradcam = deprocess_image(gb*cam3)
        guided_gradcam = cv2.cvtColor(guided_gradcam, cv2.COLOR_BGR2RGB)
        plt.imshow(guided_gradcam)
        plt.title("Guided GradCAM", fontsize=20)
        plt.axis("off")
        
        k += 3
    plt.show()

## Mise au point d'un modèle de classification binaire 

### Data generator 

In [ ]:
# Création d'un data generator pour le dataset d'entrainement
train_datagen = ImageDataGenerator(
        preprocessing_function = preprocess_input,
        rotation_range = 10,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        zoom_range = 0.1,
        horizontal_flip = True
        )

train_df["label"] = train_df["label"].apply(str)

train_generator = train_datagen.flow_from_dataframe(train_df, IMAGE_DIR,x_col="filename", y_col="label",
                                                    target_size=(W,H), class_mode="categorical",
                                                   batch_size=batch_size, shuffle=True, seed=SEED)

In [ ]:
# Création d'un data generator pour le dataset d'entrainement rééquilibré
train_datagen = ImageDataGenerator(
        preprocessing_function = preprocess_input,
        rotation_range = 10,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        zoom_range = 0.1,
        horizontal_flip = True
        )

train_df_X["label"] = train_df_X["label"].apply(str)

train_generator_X = train_datagen.flow_from_dataframe(train_df_X, IMAGE_DIR,x_col="filename", y_col="label",
                                                    target_size=(W,H), class_mode="categorical",
                                                   batch_size=batch_size, shuffle=True, seed=SEED)

In [ ]:
# Création d'un data generator pour le dataset de validation
val_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input)

val_df["label"] = val_df["label"].apply(str)
val_generator = val_datagen.flow_from_dataframe(val_df, IMAGE_DIR, x_col="filename", y_col="label",
                                               target_size=(W,H), class_mode="categorical",
                                                batch_size=batch_size)

In [ ]:
# Création d'un data generator pour le dataset de validation rééquilibré
val_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input)

val_df_X["label"] = val_df_X["label"].apply(str)
val_generator_X = val_datagen.flow_from_dataframe(val_df_X, IMAGE_DIR, x_col="filename", y_col="label",
                                               target_size=(W,H), class_mode="categorical",
                                                batch_size=batch_size)

In [ ]:
# Observation du fonctionnement du data generator sur quelques images du dataset d'entrainement
ex_df = train_df.sample(n=15).reset_index(drop=True)
ex_gen = train_datagen.flow_from_dataframe(ex_df,IMAGE_DIR,x_col="filename", y_col="label",
                                           target_size=(W,H), class_mode="categorical")

In [ ]:
plt.figure(figsize=(15,15))
for i in range(0, 9):
    plt.subplot(5,3,i+1)
    for x, y in ex_gen:
        im = x[0]
        plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        break
plt.tight_layout()
plt.show()

### Modèle 1 : modèle simple

In [ ]:
# création d'un modèle simple
simple_model = tf.keras.Sequential(name = 'simple_model')
# block1
simple_model.add(Conv2D(filters = 30,                    # Nombre de filtres
                        kernel_size = (5, 5),            # Shape du kernel
                        input_shape = (224, 224, 3),     # Shape de l'entrée
                        activation = 'relu',             # Fonction d'activation
                        name = 'conv1_block1_out'))      # nom de la couche
simple_model.add(MaxPooling2D(pool_size = (2, 2),
                              name = 'maxpo1_block1_out'))
simple_model.add(Conv2D(filters = 16,                    
                        kernel_size = (3, 3),
                        activation = 'relu',
                        name = 'conv2_block1_out'))
simple_model.add(MaxPooling2D(pool_size = (2, 2),
                              name = 'maxpo2_block1_out'))
# block2
simple_model.add(Flatten(name = 'flat1_block2_out'))
simple_model.add(Dropout(rate = 0.2,
                         name = 'drop1_block2_out'))
#block3
simple_model.add(Dense(units = 128,
                       activation = 'relu',
                       name = 'den1_block3_out'))
simple_model.add(Dense(units = 2,
                       activation = 'softmax',
                       name = 'den2_block3_out'))

simple_model.summary()

### Modéle 2 : transfert learning de EfficientNetB0 

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
efficientnet = EfficientNetB0(include_top=False, pooling="avg", weights='imagenet')
for layer in efficientnet.layers:
    layer.trainable=False

logits = Dense(2)(efficientnet.layers[-1].output)
output = Activation('softmax')(logits)
efficientnet = Model(efficientnet.input, output, name = 'LT_EfficientNetB0')
efficientnet.summary()

### Modèle 3 : transfert learning de VGG19

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
vgg19 = EfficientNetB0(include_top=False, pooling="avg", weights='imagenet')
for layer in vgg19.layers:
    layer.trainable=False

logits = Dense(2)(vgg19.layers[-1].output)
output = Activation('softmax')(logits)
vgg19 = Model(vgg19.input, output, name = 'TL_VGG19')
vgg19.summary()

### Modèle 4 : transfert learning de Xception

In [ ]:
from tensorflow.keras.applications.xception import Xception
xception = Xception(include_top=False, pooling="avg", weights='imagenet')
for layer in xception.layers:
    layer.trainable=False

logits = Dense(2)(xception.layers[-1].output)
output = Activation('softmax')(logits)
xception = Model(xception.input, output, name = 'TL_Xception')
xception.summary()

### Modèle 5 : transfert learning de ResNet50V2

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
resnet = ResNet50V2(include_top=False, pooling="avg", weights='imagenet')
for layer in resnet.layers:
    layer.trainable=False

logits = Dense(2)(resnet.layers[-1].output)
output = Activation('softmax')(logits)
resnet = Model(resnet.input, output, name = 'TL_ResNet50V2')
resnet.summary()

### Modèle 6 : transfert learning de InceptionV3

In [ ]:
from tensorflow.keras.applications import InceptionV3
inceptionV3 = InceptionV3(include_top=False, pooling="avg", weights='imagenet')
for layer in inceptionV3.layers:
    layer.trainable=False

logits = Dense(2)(inceptionV3.layers[-1].output)
output = Activation('softmax')(logits)
inceptionV3 = Model(inceptionV3.input, output, name = 'TL_InceptionV3')
inceptionV3.summary()

### Compile 

In [ ]:
sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
simple_model.compile(optimizer=sgd, loss = "categorical_crossentropy", metrics=["accuracy"])
efficientnet.compile(optimizer=sgd, loss = "categorical_crossentropy", metrics=["accuracy"])
vgg19.compile(optimizer=sgd, loss = "categorical_crossentropy", metrics=["accuracy"])
xception.compile(optimizer=sgd, loss = "categorical_crossentropy", metrics=["accuracy"])
resnet.compile(optimizer=sgd, loss = "categorical_crossentropy", metrics=["accuracy"])
inceptionV3.compile(optimizer=sgd, loss = "categorical_crossentropy", metrics=["accuracy"])


### Entrainement des modèles 

In [ ]:
# definition de callbacks :
earlystoper = EarlyStopping(monitor='val_loss', patience=3)

checkpointer_simple_model = ModelCheckpoint(filepath=f'{output_dir}/simple_model_best.hdf5', monitor='val_loss', save_best_only=True, mode='auto')
checkpointer_efficientnet = ModelCheckpoint(filepath=f'{output_dir}/TL_EfficientNetB0_best.hdf5', monitor='val_loss', save_best_only=True, mode='auto')
checkpointer_vgg19 = ModelCheckpoint(filepath=f'{output_dir}/TL_VGG19_best.hdf5', monitor='val_loss', save_best_only=True, mode='auto')
checkpointer_xception = ModelCheckpoint(filepath=f'{output_dir}/TL_Xception_best.hdf5', monitor='val_loss', save_best_only=True, mode='auto')
checkpointer_resnet = ModelCheckpoint(filepath=f'{output_dir}/TL_ResNet50V2_best.hdf5', monitor='val_loss', save_best_only=True, mode='auto')

CSV_logger_simple_model = CSVLogger(filename = 'logger_simple_model.csv', separator=',', append = True)
CSV_logger_efficientnet = CSVLogger(filename = 'logger_TL_EfficientNetB0.csv', separator=',', append = True)
CSV_logger_VGG19 = CSVLogger(filename = 'logger_TL_VGG19.csv', separator=',', append = True)
CSV_logger_xception = CSVLogger(filename = 'logger_TL_Xception.csv', separator=',', append = True)
CSV_logger_resnet = CSVLogger(filename = 'logger_TL_ResNet50V2.csv', separator=',', append = True)

callbacks_simple_model = [earlystoper, CSV_logger_simple_model, checkpointer_simple_model]
callbacks_efficientnet = [earlystoper, CSV_logger_efficientnet, checkpointer_efficientnet]
callbacks_vgg19 = [earlystoper, CSV_logger_VGG19, checkpointer_vgg19]
callbacks_xception = [earlystoper, CSV_logger_xception, checkpointer_xception]
callbacks_resnet = [earlystoper, CSV_logger_resnet, checkpointer_resnet]

In [ ]:
# a revoir....
'''#définition d'une fonction de génération de callbacks
def callbacks (model):
    earlystoper = EarlyStopping(monitor='val_loss', patience=3)
    checkpointer = ModelCheckpoint(filepath=f'{output_dir}/TL_{model}_best.hdf5', monitor='val_loss', save_best_only=True, mode='auto')
    callbacks = [earlystoper, checkpointer]
    
# définition d'une fonction permettant l'entrianement de modèles
def trainer (model, optimizer, epochs, batch_size):
    model.compile(optimizer=optimizer, loss = "categorical_crossentropy", metrics=["accuracy"])
    history = model.fit_generator(train_generator,
                                  epochs=epochs,
                                  validation_data=val_generator,
                                  validation_steps=len(val_df)//batch_size,
                                  steps_per_epoch=len(train_df)//batch_size,
                                  callbacks=callbacks(model)
                                 )
    '''

In [ ]:
# recherche de possibilité d'utilisation d'une GPU
print('GPU Available : ', tf.config.experimental.list_physical_devices('GPU'))

In [ ]:
# a revoir
'''# entrainement des modèles
optimizer = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
trainer(simple_model, optimizer, 5, 128)'''

In [ ]:
# entrainement des différents modéles
'''
history_simple_model = simple_model.fit_generator(train_generator,
                                                  epochs=epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size,
                                                  callbacks=callbacks_simple_model)

history_efficientnet = efficientnet.fit_generator(train_generator,
                                                  epochs=epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size,
                                                  callbacks=callbacks_efficientnet)

history_vgg19 = vgg19.fit_generator(train_generator,
                                                  epochs=epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size,
                                                  callbacks=callbacks_vgg19)

history_xception = xception.fit_generator(train_generator,
                                                  epochs=epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size,
                                                  callbacks=callbacks_xception)

history_resnet = resnet.fit_generator(train_generator,
                                                  epochs=epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size,
                                                  callbacks=callbacks_resnet)
'''

### Entrainement des modèles sur datasets rééquilibrés

In [ ]:
# definition de callbacks :
earlystoper = EarlyStopping(monitor='val_loss', patience=6)

checkpointer_simple_model_X = ModelCheckpoint(filepath=f'{output_dir}/simple_model_X_best.hdf5', monitor='val_loss', save_best_only=True, mode='auto')
checkpointer_efficientnet_X = ModelCheckpoint(filepath=f'{output_dir}/TL_EfficientNetB0_X_best.hdf5', monitor='val_loss', save_best_only=True, mode='auto')
checkpointer_vgg19_X = ModelCheckpoint(filepath=f'{output_dir}/TL_VGG19_X_best.hdf5', monitor='val_loss', save_best_only=True, mode='auto')
checkpointer_xception_X = ModelCheckpoint(filepath=f'{output_dir}/TL_Xception_X_best.hdf5', monitor='val_loss', save_best_only=True, mode='auto')
checkpointer_resnet_X = ModelCheckpoint(filepath=f'{output_dir}/TL_ResNet50V2_X_best.hdf5', monitor='val_loss', save_best_only=True, mode='auto')
checkpointer_InceptionV3 = ModelCheckpoint(filepath=f'{output_dir}/TL_InceptionV3_X_best.hdf5', monitor='val_loss', save_best_only=True, mode='auto')

CSV_logger_simple_model_X = CSVLogger(filename = 'logger_simple_model_X.csv', separator=',', append = True)
CSV_logger_efficientnet_X = CSVLogger(filename = 'logger_TL_EfficientNetB0_X.csv', separator=',', append = True)
CSV_logger_VGG19_X = CSVLogger(filename = 'logger_TL_VGG19_X.csv', separator=',', append = True)
CSV_logger_xception_X = CSVLogger(filename = 'logger_TL_Xception_X.csv', separator=',', append = True)
CSV_logger_resnet_X = CSVLogger(filename = 'logger_TL_ResNet50V2_X.csv', separator=',', append = True)
CSV_logger_InceptionV3_X = CSVLogger(filename = 'logger_TL_InceptionV3_X.csv', separator=',', append = True)

callbacks_simple_model_X = [earlystoper, CSV_logger_simple_model_X, checkpointer_simple_model_X]
callbacks_efficientnet_X = [earlystoper, CSV_logger_efficientnet_X, checkpointer_efficientnet_X]
callbacks_vgg19_X = [earlystoper, CSV_logger_VGG19_X, checkpointer_vgg19_X]
callbacks_xception_X = [earlystoper, CSV_logger_xception_X, checkpointer_xception_X]
callbacks_resnet_X = [earlystoper, CSV_logger_resnet_X, checkpointer_resnet_X]
callbacks_InceptionV3_X = [earlystoper, CSV_logger_InceptionV3_X, checkpointer_InceptionV3]

In [ ]:
# entrainement du modèle simple sur le jeu de données rééquilibré
history_simple_model_X = simple_model.fit_generator(train_generator_X,
                                                  epochs=epochs,
                                                  validation_data=val_generator_X,
                                                  validation_steps=len(val_df_X)//batch_size,
                                                  steps_per_epoch=len(train_df_X)//batch_size,
                                                  callbacks=callbacks_simple_model_X)

In [ ]:
# entrainement du modèle efficientnet sur le jeu de données rééquilibré
history_efficientnet_X = efficientnet.fit_generator(train_generator_X,
                                                  epochs=epochs,
                                                  validation_data=val_generator_X,
                                                  validation_steps=len(val_df_X)//batch_size,
                                                  steps_per_epoch=len(train_df_X)//batch_size,
                                                  callbacks=callbacks_efficientnet_X)

In [ ]:
# entrainement du modèle vgg19 sur le jeu de données rééquilibré
history_vgg19_X = vgg19.fit_generator(train_generator_X,
                                                  epochs=epochs,
                                                  validation_data=val_generator_X,
                                                  validation_steps=len(val_df_X)//batch_size,
                                                  steps_per_epoch=len(train_df_X)//batch_size,
                                                  callbacks=callbacks_vgg19_X)

In [ ]:
# entrainement du modèle xception sur le jeu de données rééquilibré
history_xception_X = xception.fit_generator(train_generator_X,
                                                  epochs=epochs,
                                                  validation_data=val_generator_X,
                                                  validation_steps=len(val_df_X)//batch_size,
                                                  steps_per_epoch=len(train_df_X)//batch_size,
                                                  callbacks=callbacks_xception_X)

In [ ]:
# entrainement du modèle resnet sur le jeu de données rééquilibré
history_resnet_X = resnet.fit_generator(train_generator_X,
                                                  epochs=epochs,
                                                  validation_data=val_generator_X,
                                                  validation_steps=len(val_df_X)//batch_size,
                                                  steps_per_epoch=len(train_df_X)//batch_size,
                                                  callbacks=callbacks_resnet_X)

In [ ]:
# entrainement du modèle InceptionV3 sur le jeu de données rééquilibré
history_InceptionV3_X = inceptionV3.fit_generator(train_generator_X,
                                                  epochs=epochs,
                                                  validation_data=val_generator_X,
                                                  validation_steps=len(val_df_X)//batch_size,
                                                  steps_per_epoch=len(train_df_X)//batch_size,
                                                  callbacks=callbacks_InceptionV3_X)

### Affichage des GradCAMs

#### simple_model

In [ ]:
simple_model_logit = Model(simple_model.input,simple_model.layers[-2].output)

retrained_gradCAM_simple_model = GradCAM(model=simple_model_logit, layerName="maxpo2_block1_out")
retrained_guidedBP_simple_model = GuidedBackprop(model=simple_model, layerName="maxpo2_block1_out")

data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = data_gen.flow_from_dataframe(test_df, IMAGE_DIR, x_col="filename",
                                               target_size=(W,H), class_mode=None,
                                                batch_size=1, shuffle=False)

pred = simple_model.predict_generator(test_generator, steps = len(test_generator), verbose = 1)
pred_indices = np.argmax(pred,axis=1)

results = test_df.copy()
results["pred"] = pred_indices
true_edible = list(results[(results.label == 1) & (results.pred ==1)].filename)
true_inedible = list(results[(results.label == 0) & (results.pred ==0)].filename)
wrong_class = [x for x in results.filename if x not in (true_inedible+true_edible)]

# Affichage de GradCAM pour les champignons comestibles
show_gradCAMs(simple_model, retrained_gradCAM_simple_model,retrained_guidedBP_simple_model,true_edible, n=5, decode={0:'0', 1:'1'})

#### EfficientNetB0

In [ ]:
efficientnet_logit = Model(efficientnet.input,efficientnet.layers[-2].output)

retrained_gradCAM_efficientnet = GradCAM(model=efficientnet_logit, layerName="top_conv")
retrained_guidedBP_efficientnet = GuidedBackprop(model=efficientnet, layerName="top_conv")

data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = data_gen.flow_from_dataframe(test_df, IMAGE_DIR, x_col="filename",
                                               target_size=(W,H), class_mode=None,
                                                batch_size=1, shuffle=False)

pred = efficientnet.predict_generator(test_generator, steps = len(test_generator), verbose = 1)
pred_indices = np.argmax(pred,axis=1)

results = test_df.copy()
results["pred"] = pred_indices
true_edible = list(results[(results.label == 1) & (results.pred ==1)].filename)
true_inedible = list(results[(results.label == 0) & (results.pred ==0)].filename)
wrong_class = [x for x in results.filename if x not in (true_inedible+true_edible)]

# Affichage de GradCAM pour les champignons comestibles
show_gradCAMs(efficientnet, retrained_gradCAM_efficientnet,retrained_guidedBP_efficientnet,true_edible, n=5, decode={0:'0', 1:'1'})

#### VGG19

In [ ]:
vgg19_logit = Model(vgg19.input,vgg19.layers[-2].output)

retrained_gradCAM_vgg19 = GradCAM(model=vgg19_logit, layerName="top_conv")
retrained_guidedBP_vgg19 = GuidedBackprop(model=vgg19, layerName="top_conv")

data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = data_gen.flow_from_dataframe(test_df, IMAGE_DIR, x_col="filename",
                                               target_size=(W,H), class_mode=None,
                                                batch_size=1, shuffle=False)

pred = vgg19.predict_generator(test_generator, steps = len(test_generator), verbose = 1)
pred_indices = np.argmax(pred,axis=1)

results = test_df.copy()
results["pred"] = pred_indices
true_edible = list(results[(results.label == '1') & (results.pred ==1)].filename)
true_inedible = list(results[(results.label == '0') & (results.pred ==0)].filename)
wrong_class = [x for x in results.filename if x not in (true_inedible+true_edible)]

# Affichage de GradCAM pour les champignons comestibles
show_gradCAMs(vgg19, retrained_gradCAM_vgg19,retrained_guidedBP_vgg19,true_edible, n=5, decode={0:'0', 1:'1'})

#### Xception

In [ ]:
xception_logit = Model(xception.input,xception.layers[-2].output)

retrained_gradCAM_xception = GradCAM(model=xception_logit, layerName="block14_sepconv2_act")
retrained_guidedBP_xception = GuidedBackprop(model=xception, layerName="block14_sepconv2_act")

data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = data_gen.flow_from_dataframe(test_df, IMAGE_DIR, x_col="filename",
                                               target_size=(W,H), class_mode=None,
                                                batch_size=1, shuffle=False)

pred = xception.predict_generator(test_generator, steps = len(test_generator), verbose = 1)
pred_indices = np.argmax(pred,axis=1)

results = test_df.copy()
results["pred"] = pred_indices
true_edible = list(results[(results.label == 1) & (results.pred ==1)].filename)
true_inedible = list(results[(results.label == 0) & (results.pred ==0)].filename)
wrong_class = [x for x in results.filename if x not in (true_inedible+true_edible)]

# Affichage de GradCAM pour les champignons comestibles
show_gradCAMs(xception, retrained_gradCAM_xception,retrained_guidedBP_xception,true_edible, n=5, decode={0:'0', 1:'1'})

#### ResNet50V2

In [ ]:
resnet_logit = Model(resnet.input,resnet.layers[-2].output)

retrained_gradCAM_resnet = GradCAM(model=resnet_logit, layerName="conv5_block3_out")
retrained_guidedBP_resnet = GuidedBackprop(model=resnet, layerName="conv5_block3_out")

data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = data_gen.flow_from_dataframe(test_df, IMAGE_DIR, x_col="filename",
                                               target_size=(W,H), class_mode=None,
                                                batch_size=1, shuffle=False)

pred = resnet.predict_generator(test_generator, steps = len(test_generator), verbose = 1)
pred_indices = np.argmax(pred,axis=1)

results = test_df.copy()
results["pred"] = pred_indices
true_edible = list(results[(results.label == 1) & (results.pred ==1)].filename)
true_inedible = list(results[(results.label == 0) & (results.pred ==0)].filename)
wrong_class = [x for x in results.filename if x not in (true_inedible+true_edible)]

# Affichage de GradCAM pour les champignons comestibles
show_gradCAMs(resnet, retrained_gradCAM_resnet,retrained_guidedBP_resnet,true_edible, n=5, decode={0:'0', 1:'1'})

#### InceptionV3

In [ ]:
inceptionV3_logit = Model(inceptionV3.input,inceptionV3.layers[-2].output)

retrained_gradCAM_inceptionV3 = GradCAM(model=inceptionV3_logit, layerName="conv2d_97")
retrained_guidedBP_inceptionV3 = GuidedBackprop(model=inceptionV3, layerName="conv2d_97")

data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = data_gen.flow_from_dataframe(test_df, IMAGE_DIR, x_col="filename",
                                               target_size=(W,H), class_mode=None,
                                                batch_size=1, shuffle=False)

pred = inceptionV3.predict_generator(test_generator, steps = len(test_generator), verbose = 1)
pred_indices = np.argmax(pred,axis=1)

results = test_df.copy()
results["pred"] = pred_indices
true_edible = list(results[(results.label == 1) & (results.pred ==1)].filename)
true_inedible = list(results[(results.label == 0) & (results.pred ==0)].filename)
wrong_class = [x for x in results.filename if x not in (true_inedible+true_edible)]

# Affichage de GradCAM pour les champignons comestibles
show_gradCAMs(inceptionV3, retrained_gradCAM_inceptionV3,retrained_guidedBP_inceptionV3,true_edible, n=5, decode={0:'0', 1:'1'})